In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import os as os
# Import the API key.
from config import g_key

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("Weather_Data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Cape Town,-33.9258,18.4232,60.12,85,40,5.86,ZA,2021-05-29 18:28:47
1,1,Qaanaaq,77.4840,-69.3632,31.69,95,49,5.77,GL,2021-05-29 18:31:33
2,2,Ushuaia,-54.8000,-68.3000,29.86,93,90,9.22,AR,2021-05-29 18:31:51
3,3,Namibe,-15.1961,12.1522,72.61,78,0,6.13,AO,2021-05-29 18:32:00
4,4,Husavik,66.0449,-17.3389,58.91,58,100,11.77,IS,2021-05-29 18:32:01


In [4]:
# Det the data types
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
# print(g_key)

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### 6.5.3: Get Vaction Criteria

In [12]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [14]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,Rikitea,-23.1203,-134.9692,77.09,84,100,19.42,PF,2021-05-29 18:32:01
8,8,Touros,-5.1989,-35.4608,85.10,71,4,16.02,BR,2021-05-29 18:32:02
14,14,San Andres,12.5847,-81.7006,81.54,79,20,12.68,CO,2021-05-29 18:30:05
16,16,Butaritari,3.0707,172.7902,82.69,78,100,15.79,KI,2021-05-29 18:30:52
17,17,Mecca,21.4267,39.8261,89.91,33,25,6.80,SA,2021-05-29 18:30:39
20,20,Bambous Virieux,-20.3428,57.7575,75.45,94,40,5.19,MU,2021-05-29 18:30:47
23,23,Hilo,19.7297,-155.0900,78.62,75,20,0.00,US,2021-05-29 18:32:06
26,26,Puerto Ayora,-0.7393,-90.3518,84.20,58,40,19.57,EC,2021-05-29 18:31:58
34,34,Kapaa,22.0752,-159.3190,79.12,76,20,1.01,US,2021-05-29 18:31:58
45,45,Mweka,-4.8500,21.5667,77.81,59,97,2.53,CD,2021-05-29 18:32:13


In [16]:
preferred_cities_df.count()

City_ID       171
City          171
Lat           171
Lng           171
Max Temp      171
Humidity      171
Cloudiness    171
Wind Speed    171
Country       171
Date          171
dtype: int64

### 6.5.4: Map Vacation Criteria

In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Rikitea,PF,77.09,-23.1203,-134.9692,
8,Touros,BR,85.10,-5.1989,-35.4608,
14,San Andres,CO,81.54,12.5847,-81.7006,
16,Butaritari,KI,82.69,3.0707,172.7902,
17,Mecca,SA,89.91,21.4267,39.8261,
20,Bambous Virieux,MU,75.45,-20.3428,57.7575,
23,Hilo,US,78.62,19.7297,-155.0900,
26,Puerto Ayora,EC,84.20,-0.7393,-90.3518,
34,Kapaa,US,79.12,22.0752,-159.3190,
45,Mweka,CD,77.81,-4.8500,21.5667,


In [20]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

### 6.5.4: Map Vacation Criteria

In [22]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [24]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
5,Rikitea,PF,77.09,-23.1203,-134.9692,Pension Maro'i
8,Touros,BR,85.10,-5.1989,-35.4608,INN NEW HORIZON
14,San Andres,CO,81.54,12.5847,-81.7006,Hotel Arena Blanca
16,Butaritari,KI,82.69,3.0707,172.7902,Isles Sunset Lodge
17,Mecca,SA,89.91,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [32]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [34]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))